In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [19]:
class wiki_scrape:

    def __init__(self):
        self.all_links = []

    def scrape_wiki(self, url, degree):
        data = requests.get(url).content
        soup = BeautifulSoup(data, 'html.parser')
        bodyContent = soup.find("div", {"id": "bodyContent"})
        temp_link_list = []
        for hyperlink in bodyContent.find_all('a'):
            link = hyperlink.get('href')
            if isinstance(link, str) and link[:5]=='/wiki':
                temp_link = f'https://en.wikipedia.org{link}'
                self.all_links.append([url, temp_link])
                temp_link_list.append(temp_link)
        degree += 1
        
        if degree<15:
            for url in temp_link_list:
                self.scrape_wiki(url, degree)
        

In [ ]:
obj = wiki_scrape()
obj.scrape_wiki('https://en.wikipedia.org/wiki/Earl_of_Dysart', 1)
pd.DataFrame(obj.all_links)

In [ ]:
df = pd.DataFrame(obj.all_links)
df.columns = ['source', 'destination']

In [ ]:
df.to_csv('wiki_links.csv', index=False)